# The Currencies data-set

The R version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-R/Currencies.ipynb)

In [1]:
import pandas as pd    
from sqlalchemy import func, and_, or_, text, Integer
from sqlalchemy.orm import aliased
from sqlalchemy.sql.expression import cast
from plutoPy.model import RoutingSession, Currencies
from datetime import date, datetime, timedelta

### get traded futures pairs

In [2]:
results = (RoutingSession.session.query(Currencies.NseFuturesTimeSeries.SYMBOL, 
                                        func.min(Currencies.NseFuturesTimeSeries.TIME_STAMP).label('start_dt'), 
                                        func.max(Currencies.NseFuturesTimeSeries.TIME_STAMP).label('end_dt'))
            .group_by(Currencies.NseFuturesTimeSeries.SYMBOL)
            .order_by(text('start_dt'))
            .all())

for instance in results:
    print(instance)

('EURINR', datetime.date(2010, 10, 29), datetime.date(2019, 7, 22))
('GBPINR', datetime.date(2010, 10, 29), datetime.date(2019, 7, 22))
('JPYINR', datetime.date(2010, 10, 29), datetime.date(2019, 7, 22))
('USDINR', datetime.date(2010, 10, 29), datetime.date(2019, 7, 22))
('USDJPY', datetime.date(2018, 2, 27), datetime.date(2019, 7, 22))
('GBPUSD', datetime.date(2018, 2, 27), datetime.date(2019, 7, 22))
('EURUSD', datetime.date(2018, 2, 27), datetime.date(2019, 7, 22))


/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)


### get the latest USDINR option chain for the nearest expiry   

In [3]:
end_dt = RoutingSession.session.query(func.max(Currencies.NseOptionsTimeSeries.TIME_STAMP)).scalar() 

expiry = (RoutingSession.session.query(func.min(Currencies.NseOptionsTimeSeries.EXPIRY))
          .filter(and_(Currencies.NseOptionsTimeSeries.SYMBOL == "USDINR",
                       Currencies.NseOptionsTimeSeries.TIME_STAMP == end_dt))
          .scalar())

results = (RoutingSession.session.query(Currencies.NseOptionsTimeSeries)
      .filter(and_(Currencies.NseOptionsTimeSeries.SYMBOL == "USDINR", 
                   Currencies.NseOptionsTimeSeries.EXPIRY == expiry,
                   Currencies.NseOptionsTimeSeries.TIME_STAMP == end_dt))
      .order_by(Currencies.NseOptionsTimeSeries.STRIKE, Currencies.NseOptionsTimeSeries.OTYPE)
      .all())

for instance in results:
    print(instance)

USDINR(2019-Jul-26)/65.25CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/65.25PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/65.5CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/65.5PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/65.75CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/65.75PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.0CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.0PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.25CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.25PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.5CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.5PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.75CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/66.75PE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/67.0CE/2019-Jul-22: 0.0,0.0,0.0,None,0.0,0
USDINR(2019-Jul-26)/67.0PE/2019-Jul-22: 0.0,0.0

### get the currencies tracked by AlphaVantage end-of-day

In [4]:
results = (RoutingSession.session.query(Currencies.AvEodTimeSeries.SYMBOL, 
                                        func.min(Currencies.AvEodTimeSeries.TIME_STAMP).label('start_dt'), 
                                        func.max(Currencies.AvEodTimeSeries.TIME_STAMP).label('end_dt'))
            .group_by(Currencies.AvEodTimeSeries.SYMBOL)
            .order_by(text('start_dt'))
            .all())

for instance in results:
    print(instance)

('MYR', datetime.date(1998, 1, 23), datetime.date(2019, 7, 23))
('TWD', datetime.date(2000, 4, 24), datetime.date(2019, 7, 23))
('JPY', datetime.date(2000, 4, 27), datetime.date(2019, 7, 23))
('CHF', datetime.date(2000, 7, 19), datetime.date(2019, 7, 23))
('CAD', datetime.date(2000, 7, 27), datetime.date(2019, 7, 23))
('HKD', datetime.date(2000, 8, 4), datetime.date(2019, 7, 23))
('SGD', datetime.date(2000, 8, 10), datetime.date(2019, 7, 23))
('CNY', datetime.date(2005, 11, 9), datetime.date(2019, 7, 23))
('MOP', datetime.date(2011, 4, 7), datetime.date(2019, 7, 23))
('BHD', datetime.date(2014, 11, 7), datetime.date(2019, 7, 23))
('BIF', datetime.date(2014, 11, 7), datetime.date(2019, 7, 23))
('BND', datetime.date(2014, 11, 7), datetime.date(2019, 7, 23))
('BOB', datetime.date(2014, 11, 7), datetime.date(2019, 7, 23))
('BRL', datetime.date(2014, 11, 7), datetime.date(2019, 7, 23))
('BSD', datetime.date(2014, 11, 7), datetime.date(2019, 7, 23))
('BWP', datetime.date(2014, 11, 7), dateti

### get the currencies tracked by AlphaVantage 30-min bars

In [5]:
results = (RoutingSession.session.query(Currencies.Av30minTimeSeries.SYMBOL, 
                                        func.min(Currencies.Av30minTimeSeries.TIME_STAMP).label('start_dt'), 
                                        func.max(Currencies.Av30minTimeSeries.TIME_STAMP).label('end_dt'),
                                        func.count(Currencies.Av30minTimeSeries.TIME_STAMP).label('count'))
            .group_by(Currencies.Av30minTimeSeries.SYMBOL)
            .order_by(text('count desc'))
            .all())

for instance in results:
    print(instance)            
    

('AED', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 6, 30), 14362)
('AFN', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 4, 0), 14354)
('ANG', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 7, 30), 14286)
('ARS', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 4, 0), 14226)
('AWG', datetime.datetime(2018, 8, 21, 17, 0), datetime.datetime(2019, 7, 23, 7, 30), 14076)
('AUD', datetime.datetime(2018, 8, 23, 4, 0), datetime.datetime(2019, 7, 23, 10, 30), 14045)
('BAM', datetime.datetime(2018, 8, 23, 4, 0), datetime.datetime(2019, 7, 23, 10, 30), 13402)
('BDT', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 10, 30), 13195)
('BBD', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 10, 0), 13161)
('SDG', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7, 23, 8, 0), 13113)
('EGP', datetime.datetime(2018, 8, 21, 16, 0), datetime.datetime(2019, 7

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)